In [181]:
import pandas as pd
import numpy as np
import yfinance as yf
AAPL = yf.Ticker("AAPL")
opt = AAPL.option_chain('2020-07-16')
calls = opt.calls
puts = opt.puts
# print(calls)


In [182]:
k = calls[calls['inTheMoney'] == False]
l = k[['strike', 'lastPrice']]
m = l.set_index('strike').to_dict()
n = m['lastPrice'] 
print(n)

{385.0: 5.77, 387.5: 4.7, 390.0: 3.8, 392.5: 3.03, 395.0: 2.42, 397.5: 1.9, 400.0: 1.54, 405.0: 0.99, 410.0: 0.68, 415.0: 0.48, 420.0: 0.37, 425.0: 0.28, 430.0: 0.23, 435.0: 0.19, 440.0: 0.14, 445.0: 0.12, 450.0: 0.1, 455.0: 0.09, 460.0: 0.07, 465.0: 0.06, 470.0: 0.04, 475.0: 0.04, 480.0: 0.03, 485.0: 0.03, 490.0: 0.03, 495.0: 0.03, 500.0: 0.02, 505.0: 0.02, 510.0: 0.02, 515.0: 0.02, 520.0: 0.02, 525.0: 0.01, 530.0: 0.01, 535.0: 0.02, 540.0: 0.01}


In [185]:
def Call_Credit_Spread():
#     Credit spreads should be otm
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio
#     Change currentRatio with the best available one and save that transcation
    
    for x in n:
        for k in n:
            short = 1
            long = 2
#             finalSpread = [short, long, ratio]
            currentRatio = 1
            strikeShort = x
            strikeLong = k
            #Selling the cheapest one and then buying the more expensive one, one by one
            premiumShort = n[x]
            premiumLong = n[k]
            #figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
            MaxReward = premiumShort - premiumLong
            print("MaxReward = ", MaxReward)
            MaxRisk = strikeLong - strikeShort - MaxReward
            print("MaxRisk = ", MaxRisk)
            if MaxReward == 0:
                Ratio = 1
            else: 
                Ratio = MaxRisk/MaxReward
            print("Ratio = ", Ratio)
            if Ratio < currentRatio:
                currentRatio = Ratio
                short = strikeShort
                long = strikeLong
#                 print("currentRatio = ", currentRatio)
#     return(short, long, currentRatio)
                    
        



In [186]:
print(Call_Credit_Spread())


MaxReward =  0.0
MaxRisk =  0.0
Ratio =  1
MaxReward =  1.0699999999999994
MaxRisk =  1.4300000000000006
Ratio =  1.3364485981308425
MaxReward =  1.9699999999999998
MaxRisk =  3.0300000000000002
Ratio =  1.538071065989848
MaxReward =  2.7399999999999998
MaxRisk =  4.76
Ratio =  1.7372262773722629
MaxReward =  3.3499999999999996
MaxRisk =  6.65
Ratio =  1.985074626865672
MaxReward =  3.8699999999999997
MaxRisk =  8.63
Ratio =  2.229974160206719
MaxReward =  4.2299999999999995
MaxRisk =  10.77
Ratio =  2.546099290780142
MaxReward =  4.779999999999999
MaxRisk =  15.22
Ratio =  3.1841004184100425
MaxReward =  5.09
MaxRisk =  19.91
Ratio =  3.911591355599214
MaxReward =  5.289999999999999
MaxRisk =  24.71
Ratio =  4.671077504725899
MaxReward =  5.3999999999999995
MaxRisk =  29.6
Ratio =  5.481481481481482
MaxReward =  5.489999999999999
MaxRisk =  34.51
Ratio =  6.285974499089254
MaxReward =  5.539999999999999
MaxRisk =  39.46
Ratio =  7.12274368231047
MaxReward =  5.579999999999999
MaxRisk 

MaxReward =  -2.3499999999999996
MaxRisk =  -15.15
Ratio =  6.446808510638299
MaxReward =  -1.7399999999999998
MaxRisk =  -13.26
Ratio =  7.620689655172415
MaxReward =  -1.2199999999999998
MaxRisk =  -11.280000000000001
Ratio =  9.245901639344265
MaxReward =  -0.86
MaxRisk =  -9.14
Ratio =  10.627906976744187
MaxReward =  -0.30999999999999994
MaxRisk =  -4.69
Ratio =  15.12903225806452
MaxReward =  0.0
MaxRisk =  0.0
Ratio =  1
MaxReward =  0.20000000000000007
MaxRisk =  4.8
Ratio =  23.999999999999993
MaxReward =  0.31000000000000005
MaxRisk =  9.69
Ratio =  31.258064516129025
MaxReward =  0.4
MaxRisk =  14.6
Ratio =  36.5
MaxReward =  0.45000000000000007
MaxRisk =  19.55
Ratio =  43.44444444444444
MaxReward =  0.49000000000000005
MaxRisk =  24.51
Ratio =  50.0204081632653
MaxReward =  0.54
MaxRisk =  29.46
Ratio =  54.55555555555555
MaxReward =  0.56
MaxRisk =  34.44
Ratio =  61.49999999999999
MaxReward =  0.5800000000000001
MaxRisk =  39.42
Ratio =  67.9655172413793
MaxReward =  0.5

Ratio =  403.76190476190476
MaxReward =  0.21000000000000002
MaxRisk =  89.79
Ratio =  427.57142857142856
MaxReward =  0.22
MaxRisk =  94.78
Ratio =  430.8181818181818
MaxReward =  0.22
MaxRisk =  99.78
Ratio =  453.54545454545456
MaxReward =  0.21000000000000002
MaxRisk =  104.79
Ratio =  499.0
MaxReward =  0.22
MaxRisk =  109.78
Ratio =  499.0
MaxReward =  -5.579999999999999
MaxRisk =  -44.42
Ratio =  7.9605734767025105
MaxReward =  -4.51
MaxRisk =  -42.99
Ratio =  9.532150776053216
MaxReward =  -3.61
MaxRisk =  -41.39
Ratio =  11.465373961218837
MaxReward =  -2.84
MaxRisk =  -39.66
Ratio =  13.964788732394366
MaxReward =  -2.23
MaxRisk =  -37.77
Ratio =  16.937219730941706
MaxReward =  -1.71
MaxRisk =  -35.79
Ratio =  20.92982456140351
MaxReward =  -1.35
MaxRisk =  -33.65
Ratio =  24.925925925925924
MaxReward =  -0.8
MaxRisk =  -29.2
Ratio =  36.5
MaxReward =  -0.49000000000000005
MaxRisk =  -24.51
Ratio =  50.0204081632653
MaxReward =  -0.29
MaxRisk =  -19.71
Ratio =  67.9655172413

Ratio =  1499.0000000000002
MaxReward =  -5.7299999999999995
MaxRisk =  -79.27
Ratio =  13.834205933682373
MaxReward =  -4.66
MaxRisk =  -77.84
Ratio =  16.703862660944207
MaxReward =  -3.76
MaxRisk =  -76.24
Ratio =  20.27659574468085
MaxReward =  -2.9899999999999998
MaxRisk =  -74.51
Ratio =  24.919732441471577
MaxReward =  -2.38
MaxRisk =  -72.62
Ratio =  30.51260504201681
MaxReward =  -1.8599999999999999
MaxRisk =  -70.64
Ratio =  37.97849462365592
MaxReward =  -1.5
MaxRisk =  -68.5
Ratio =  45.666666666666664
MaxReward =  -0.95
MaxRisk =  -64.05
Ratio =  67.42105263157895
MaxReward =  -0.64
MaxRisk =  -59.36
Ratio =  92.75
MaxReward =  -0.44
MaxRisk =  -54.56
Ratio =  124.0
MaxReward =  -0.33
MaxRisk =  -49.67
Ratio =  150.5151515151515
MaxReward =  -0.24000000000000002
MaxRisk =  -44.76
Ratio =  186.49999999999997
MaxReward =  -0.19
MaxRisk =  -39.81
Ratio =  209.5263157894737
MaxReward =  -0.15
MaxRisk =  -34.85
Ratio =  232.33333333333334
MaxReward =  -0.1
MaxRisk =  -29.9
Rati

MaxRisk =  9.99
Ratio =  999.0000000000002
MaxReward =  0.009999999999999998
MaxRisk =  14.99
Ratio =  1499.0000000000002
MaxReward =  0.009999999999999998
MaxRisk =  19.99
Ratio =  1999.0000000000002
MaxReward =  0.009999999999999998
MaxRisk =  24.99
Ratio =  2499.0
MaxReward =  0.009999999999999998
MaxRisk =  29.99
Ratio =  2999.0000000000005
MaxReward =  0.019999999999999997
MaxRisk =  34.98
Ratio =  1749.0
MaxReward =  0.019999999999999997
MaxRisk =  39.98
Ratio =  1999.0000000000002
MaxReward =  0.009999999999999998
MaxRisk =  44.99
Ratio =  4499.000000000001
MaxReward =  0.019999999999999997
MaxRisk =  49.98
Ratio =  2499.0
MaxReward =  -5.739999999999999
MaxRisk =  -104.26
Ratio =  18.163763066202094
MaxReward =  -4.67
MaxRisk =  -102.83
Ratio =  22.019271948608136
MaxReward =  -3.77
MaxRisk =  -101.23
Ratio =  26.851458885941646
MaxReward =  -3.0
MaxRisk =  -99.5
Ratio =  33.166666666666664
MaxReward =  -2.39
MaxRisk =  -97.61
Ratio =  40.84100418410041
MaxReward =  -1.86999999

Ratio =  2999.0000000000005
MaxReward =  -0.009999999999999998
MaxRisk =  -24.99
Ratio =  2499.0
MaxReward =  0.0
MaxRisk =  -20.0
Ratio =  1
MaxReward =  0.0
MaxRisk =  -15.0
Ratio =  1
MaxReward =  0.0
MaxRisk =  -10.0
Ratio =  1
MaxReward =  0.0
MaxRisk =  -5.0
Ratio =  1
MaxReward =  0.0
MaxRisk =  0.0
Ratio =  1
MaxReward =  0.01
MaxRisk =  4.99
Ratio =  499.0
MaxReward =  0.01
MaxRisk =  9.99
Ratio =  999.0
MaxReward =  0.0
MaxRisk =  15.0
Ratio =  1
MaxReward =  0.01
MaxRisk =  19.99
Ratio =  1998.9999999999998
MaxReward =  -5.76
MaxRisk =  -134.24
Ratio =  23.305555555555557
MaxReward =  -4.69
MaxRisk =  -132.81
Ratio =  28.317697228144986
MaxReward =  -3.79
MaxRisk =  -131.21
Ratio =  34.62005277044855
MaxReward =  -3.02
MaxRisk =  -129.48
Ratio =  42.87417218543046
MaxReward =  -2.41
MaxRisk =  -127.59
Ratio =  52.941908713692946
MaxReward =  -1.89
MaxRisk =  -125.61
Ratio =  66.46031746031747
MaxReward =  -1.53
MaxRisk =  -123.47
Ratio =  80.69934640522875
MaxReward =  -0.98